Berkshire Hathaway 13F: https://www.sec.gov/Archives/edgar/data/1067983/000095012322012275/18337.xml

In [3]:
# import libraries
import pandas as pd
import xml.etree.ElementTree as et
import numpy as np

In [8]:
latest_file = 'latest.xml'
prior_file = 'prior.xml'

In [9]:
def get_data(file):
    tree = et.parse(file)
    root = tree.getroot()
    ns = {'info': 'http://www.sec.gov/edgar/document/thirteenf/informationtable'}
    full_list = []
    for stocks in root.findall('info:infoTable', ns):
        issuer = stocks.find('info:nameOfIssuer', ns).text
        sec_type = stocks.find('info:titleOfClass', ns).text
        cusip = stocks.find('info:cusip', ns).text
        val = int(stocks.find('info:value', ns).text)
        # run subquery on stocks with different namespace
        shares = stocks.find('info:shrsOrPrnAmt',ns)
        amt = int(shares[0].text)
        full_list.append([issuer,sec_type,cusip,amt,val])
    df = pd.DataFrame(full_list,columns=['issuer','security_type','cusip','amount','value'])
    # create pivot table
    table = pd.pivot_table(df,index=['issuer','security_type','cusip'],aggfunc={'amount': np.sum,'value': np.sum})
    return table

In [10]:
df1 = get_data(latest_file)
df2 = get_data(prior_file)
df3 = df1.merge(df2,on=['issuer','security_type','cusip'],how='outer',suffixes=('_latest', '_prior'))

In [11]:
df3

,,,amount_latest,value_latest,amount_prior,value_prior
issuer,security_type,cusip,,,,
ACTIVISION BLIZZARD INC,COM,00507V109,5.271708e+07,4.035492e+09,6.014187e+07,4470946.0
ALLY FINL INC,COM,02005N100,2.980000e+07,7.286100e+08,3.000000e+07,834901.0
AMAZON COM INC,COM,023135106,1.066600e+07,8.959440e+08,1.066600e+07,1205258.0
AMERICAN EXPRESS CO,COM,025816109,1.516107e+08,2.240048e+10,1.516107e+08,20453800.0
AON PLC,SHS CL A,G0403H108,4.396000e+06,1.319415e+09,4.396000e+06,1177556.0
APPLE INC,COM,037833100,8.951362e+08,1.163050e+11,8.948023e+08,123661679.0
BANK AMER CORP,COM,060505104,1.010101e+09,3.345453e+10,1.010101e+09,30505039.0
BANK OF NEW YORK MELLON CORP,COM,064058100,2.506987e+07,1.141180e+09,6.221088e+07,2396364.0
CELANESE CORP DEL,COM,150870103,9.710183e+06,9.927691e+08,9.710183e+06,877219.0


# This part of the docs was helpful in refactoring my XML parsing code
## Parsing XML with Namespaces  

https://docs.python.org/3/library/xml.etree.elementtree.html  

If the XML input has namespaces, tags and attributes with prefixes in the form prefix:sometag get expanded to {uri}sometag where the prefix is replaced by the full URI. Also, if there is a default namespace, that full URI gets prepended to all of the non-prefixed tags.  

Here is an XML example that incorporates two namespaces, one with the prefix “fictional” and the other serving as the default namespace:  

<?xml version="1.0"?>
<actors xmlns:fictional="http://characters.example.com"
        xmlns="http://people.example.com">
    <actor>
        <name>John Cleese</name>
        <fictional:character>Lancelot</fictional:character>
        <fictional:character>Archie Leach</fictional:character>
    </actor>
    <actor>
        <name>Eric Idle</name>
        <fictional:character>Sir Robin</fictional:character>
        <fictional:character>Gunther</fictional:character>
        <fictional:character>Commander Clement</fictional:character>
    </actor>
</actors>  

One way to search and explore this XML example is to manually add the URI to every tag or attribute in the xpath of a find() or findall():  

root = fromstring(xml_text)
for actor in
root.findall('{http://people.example.com}actor'):
    name = actor.find('{http://people.example.com}name')
    print(name.text)
    for char in actor.findall('{http://characters.example.com}character'):
        print(' |-->', char.text)  
        
A better way to search the namespaced XML example is to create a dictionary with your own prefixes and use those in the search functions:  

ns = {'real_person': 'http://people.example.com',
      'role': 'http://characters.example.com'}

for actor in root.findall('real_person:actor', ns):
    name = actor.find('real_person:name', ns)
    print(name.text)
    for char in actor.findall('role:character', ns):
        print(' |-->', char.text)